**Installing the Hugging Face Transformers Library**

First, i have the  to install Hugging Face Transformers library. I can install Hugging Face Transformers library using pip:

In [1]:
#nstall necessary libraries

!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [5]:
!pip install -q torch torchvision transformers

 **Importing Libraries and Loading Pre-trained Models**

I'll import the necessary libraries and load a smaller pre-trained model, distilbert-base-uncased, along with its tokenizer.

In [6]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification
import torch

# Load pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Sample text
text = "Hello, HuggingFace!"

# Encode the input text
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  7592,  1010, 17662, 12172,   999,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


**Obtaining Model Outputs**

Next, I'll pass the encoded input through the model to get the outputs.

In [7]:
# Perform inference without gradient calculation
with torch.no_grad():
    output = model(**encoded_input)

# Extract the sequence output (representations for each token in the input)
sequence_output = output.last_hidden_state
print(sequence_output)

tensor([[[-0.2196, -0.0322,  0.0999,  ..., -0.1692,  0.3533,  0.3348],
         [-0.2579,  0.1615,  0.5010,  ..., -0.1564,  0.6251,  0.0260],
         [-0.6412,  0.4059,  0.5293,  ..., -0.4974,  0.1799,  0.2942],
         ...,
         [ 0.3034,  0.2239,  0.6625,  ..., -0.0076,  0.1715, -0.0462],
         [-0.2978, -0.1648,  0.0672,  ...,  0.1720,  0.4273, -0.0724],
         [ 0.8863,  0.3219, -0.4423,  ...,  0.2777, -0.4347, -0.2798]]])


**Sequence Classification with DistilBERT**

Now, i will load the DistilBertForSequenceClassification model and perform a simple classification task.

In [8]:
# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Example input texts
texts = ["Hello, HuggingFace!", "Transformers are awesome!"]

# Tokenize and prepare inputs
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
outputs = model(**encoded_inputs)

# Get the logits from the model's output
logits = outputs.logits
print(logits)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.0611, -0.0710],
        [-0.0805, -0.0850]], grad_fn=<AddmmBackward0>)


** Fine-Tuning for a Custom Classification Task**

I'll create a small dataset, tokenize the data, and fine-tune the model.

In [9]:
# Dummy dataset
texts = ["NLP is best.", "I don't like this library."]
labels = torch.tensor([1, 0])  # 1: Positive, 0: Negative

# Tokenize the dataset
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
encoded_inputs["labels"] = labels

# Fine-tuning the model
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
model.train()

# Training loop
for epoch in range(3):
    optimizer.zero_grad()
    outputs = model(**encoded_inputs)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.6864511966705322
Epoch 2, Loss: 0.643436074256897
Epoch 3, Loss: 0.6362118124961853


**Conclusion**

By following these steps, we can effectively utilize pre-trained models from the Hugging Face library for various NLP tasks and even fine-tune them for specific needs.